In [1]:
import numpy as np
import librosa
import pygame
from pygame.locals import *
import os

def clamp(min_value, max_value, value):
    return max(min(value, max_value), min_value)

target_frequency = (158, 365)  # 包括中央C的頻率範圍
filename = r"C:\Users\garyhu\Desktop\鋼琴.mp3"
output_dir = 'frames'  # 保存帧的目录

# 创建保存帧的目录
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 載入音頻檔案
audio, sr = librosa.load(filename, sr=None)

# 執行短時傅立葉變換    
stft = np.abs(librosa.stft(audio, hop_length=512, n_fft=2048 * 8))  

# 初始化 pygame
pygame.init()

# 設置視窗大小
screen_width = 1920
screen_height = 1080
screen = pygame.display.set_mode((screen_width, screen_height))

# 設定震幅的最大值和最小值
max_amplitude = np.max(stft)
min_amplitude = np.min(stft)

# 定義每一幀的長度（以毫秒為單位）
frame_duration_ms = 10

# 設置時鐘對象
clock = pygame.time.Clock()

# 設置音頻
pygame.mixer.init()
pygame.mixer.music.load(filename)

# 開始播放音頻
pygame.mixer.music.play()

# 设置帧率
fps = 1000 // frame_duration_ms

# 遍歷 stft 的每一幀，將其轉換為影片中的一幀並播放
for i in range(stft.shape[1]):  # 使用 stft.shape[1] 作為時間軸的大小
    screen.fill((255, 255, 255))  # 清空屏幕，填充為白色

    # 縮放震幅到屏幕範圍內
    scaled_stft = ((stft[:, i] - min_amplitude) / (max_amplitude - min_amplitude)) * (screen_height - 100) + 50

    # 將 stft 的每一列作為一個折線圖顯示在屏幕上
    points = [(j * screen_width // len(stft), screen_height - int(scaled_stft[j])) for j in range(stft.shape[0])]  # 構建折線圖的點
    pygame.draw.lines(screen, (255, 0, 0), False, points, 4)  # 繪製折線圖

    # 保存当前帧
    frame_filename = os.path.join(output_dir, f"frame{i:04d}.png")
    pygame.image.save(screen, frame_filename)

    pygame.display.flip()
    clock.tick(fps)

    # 監聽事件，比如點擊關閉按鈕等
    for event in pygame.event.get():
        if event.type == QUIT:
            pygame.quit()
            exit()

# 結束 pygame
pygame.quit()

# 使用 ffmpeg 将帧和音频合成为视频
os.system(f"ffmpeg -y -r {fps} -f image2 -s 1920x1080 -i {output_dir}/frame%04d.png -i {filename} -c:v libx264 -pix_fmt yuv420p movie.mp4")


pygame 2.5.2 (SDL 2.28.3, Python 3.12.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


1